In [1]:
#
# Exhaustive Search Results
#
import os
import re
import sys
import qlib
import pprint
import numpy as np
import pandas as pd

from pathlib import Path

__file__ = os.path.dirname(os.path.realpath("__file__"))
root_dir = (Path(__file__).parent / "..").resolve()
lib_dir = (root_dir / "lib").resolve()
print("The root path: {:}".format(root_dir))
print("The library path: {:}".format(lib_dir))
assert lib_dir.exists(), "{:} does not exist".format(lib_dir)
if str(lib_dir) not in sys.path:
    sys.path.insert(0, str(lib_dir))

import qlib
from qlib import config as qconfig
from qlib.workflow import R
qlib.init(provider_uri='~/.qlib/qlib_data/cn_data', region=qconfig.REG_CN)

The root path: /Users/xuanyidong/Desktop/AutoDL-Projects
The library path: /Users/xuanyidong/Desktop/AutoDL-Projects/lib


[70148:MainThread](2021-04-12 13:23:30,262) INFO - qlib.Initialization - [config.py:276] - default_conf: client.
[70148:MainThread](2021-04-12 13:23:30,266) WARNING - qlib.Initialization - [config.py:291] - redis connection failed(host=127.0.0.1 port=6379), cache will not be used!
[70148:MainThread](2021-04-12 13:23:30,269) INFO - qlib.Initialization - [__init__.py:46] - qlib successfully initialized based on client settings.
[70148:MainThread](2021-04-12 13:23:30,271) INFO - qlib.Initialization - [__init__.py:47] - data_path=/Users/xuanyidong/.qlib/qlib_data/cn_data


In [2]:
from utils.qlib_utils import QResult

In [3]:
def filter_finished(recorders):
    returned_recorders = dict()
    not_finished = 0
    for key, recorder in recorders.items():
        if recorder.status == "FINISHED":
            returned_recorders[key] = recorder
        else:
            not_finished += 1
    return returned_recorders, not_finished

def query_info(save_dir, verbose, name_filter, key_map):
    if isinstance(save_dir, list):
        results = []
        for x in save_dir:
            x = query_info(x, verbose, name_filter, key_map)
            results.extend(x)
        return results
    # Here, the save_dir must be a string
    R.set_uri(str(save_dir))
    experiments = R.list_experiments()

    if verbose:
        print("There are {:} experiments.".format(len(experiments)))
    qresults = []
    for idx, (key, experiment) in enumerate(experiments.items()):
        if experiment.id == "0":
            continue
        if name_filter is not None and re.fullmatch(name_filter, experiment.name) is None:
            continue
        recorders = experiment.list_recorders()
        recorders, not_finished = filter_finished(recorders)
        if verbose:
            print(
                "====>>>> {:02d}/{:02d}-th experiment {:9s} has {:02d}/{:02d} finished recorders.".format(
                    idx + 1,
                    len(experiments),
                    experiment.name,
                    len(recorders),
                    len(recorders) + not_finished,
                )
            )
        result = QResult(experiment.name)
        for recorder_id, recorder in recorders.items():
            result.update(recorder.list_metrics(), key_map)
            result.append_path(
                os.path.join(recorder.uri, recorder.experiment_id, recorder.id)
            )
        if not len(result):
            print("There are no valid recorders for {:}".format(experiment))
            continue
        else:
            if verbose:
                print(
                    "There are {:} valid recorders for {:}".format(
                        len(recorders), experiment.name
                    )
                )
        qresults.append(result)
    return qresults

In [4]:
paths = [root_dir / 'outputs' / 'qlib-baselines-csi300']
paths = [path.resolve() for path in paths]
print(paths)

key_map = dict()
for xset in ("train", "valid", "test"):
    key_map["{:}-mean-IC".format(xset)] = "IC ({:})".format(xset)
    key_map["{:}-mean-ICIR".format(xset)] = "ICIR ({:})".format(xset)
qresults = query_info(paths, False, 'TSF-.*-drop0_0', key_map)

[70148:MainThread](2021-04-12 13:23:31,137) INFO - qlib.workflow - [expm.py:290] - <mlflow.tracking.client.MlflowClient object at 0x7f8c4a47efa0>


[PosixPath('/Users/xuanyidong/Desktop/AutoDL-Projects/outputs/qlib-baselines-csi300')]


In [5]:
import matplotlib
from matplotlib import cm
matplotlib.use("agg")
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [6]:
def vis_depth_channel(qresults, save_path):
    save_dir = (save_path / '..').resolve()
    save_dir.mkdir(parents=True, exist_ok=True)
    print('There are {:} qlib-results'.format(len(qresults)))
    
    dpi, width, height = 200, 4000, 2000
    figsize = width / float(dpi), height / float(dpi)
    LabelSize, LegendFontsize = 22, 12
    font_gap = 5
    
    fig = plt.figure(figsize=figsize)
    # fig, axs = plt.subplots(1, 2, figsize=figsize, projection='3d')
    
    def plot_ax(cur_ax, train_or_test):
        depths, channels = [], []
        ic_values, xmaps = [], dict()
        for qresult in qresults:
            name = qresult.name.split('-')[1]
            depths.append(float(name.split('x')[0]))
            channels.append(float(name.split('x')[1]))
            if train_or_test:
                ic_values.append(qresult['IC (train)'])
            else:
                ic_values.append(qresult['IC (valid)'])
            xmaps[(depths[-1], channels[-1])] = ic_values[-1]
        # cur_ax.scatter(depths, channels, ic_values, marker='o', c="tab:orange")
        raw_depths = np.arange(1, 9, dtype=np.int32)
        raw_channels = np.array([6, 12, 24, 32, 48, 64], dtype=np.int32)
        depths, channels = np.meshgrid(raw_depths, raw_channels)
        ic_values = np.sin(depths)  # initialize
        # print(ic_values.shape)
        num_x, num_y = ic_values.shape
        for i in range(num_x):
            for j in range(num_y):
                xkey = (int(depths[i][j]), int(channels[i][j]))
                if xkey not in xmaps:
                    raise ValueError("Did not find {:}".format(xkey))
                ic_values[i][j] = xmaps[xkey]
        #print(sorted(list(xmaps.keys())))
        #surf = cur_ax.plot_surface(
        #    np.array(depths), np.array(channels), np.array(ic_values),
        #    cmap=cm.coolwarm, linewidth=0, antialiased=False)
        surf = cur_ax.plot_surface(
            depths, channels, ic_values,
            cmap=cm.Spectral, linewidth=0.2, antialiased=True)
        cur_ax.set_xticks(raw_depths)
        cur_ax.set_yticks(raw_channels)
        cur_ax.set_zticks(np.arange(4, 11, 2))
        cur_ax.set_xlabel("#depth", fontsize=LabelSize)
        cur_ax.set_ylabel("#channels", fontsize=LabelSize)
        cur_ax.set_zlabel("{:} IC (%)".format('training' if train_or_test else 'validation'), fontsize=LabelSize)
        for tick in cur_ax.xaxis.get_major_ticks():
            tick.label.set_fontsize(LabelSize - font_gap)
        for tick in cur_ax.yaxis.get_major_ticks():
            tick.label.set_fontsize(LabelSize - font_gap)
        for tick in cur_ax.zaxis.get_major_ticks():
            tick.label.set_fontsize(LabelSize - font_gap)
        # Add a color bar which maps values to colors.
#         cax = fig.add_axes([cur_ax.get_position().x1 + 0.01,
#                             cur_ax.get_position().y0,
#                             0.01,
#                             cur_ax.get_position().height * 0.9])
        # fig.colorbar(surf, cax=cax)
        # fig.colorbar(surf, shrink=0.5, aspect=5)
        # import pdb; pdb.set_trace()
        # ax1.legend(loc=4, fontsize=LegendFontsize)
    ax = fig.add_subplot(1, 2, 1, projection='3d')
    plot_ax(ax, True)
    ax = fig.add_subplot(1, 2, 2, projection='3d')
    plot_ax(ax, False)
    # fig.tight_layout()
    plt.subplots_adjust(wspace=0.05)#, hspace=0.4)
    fig.savefig(save_path, dpi=dpi, bbox_inches="tight", format="pdf")
    plt.close("all")

In [7]:
# Visualization
home_dir = Path.home()
desktop_dir = home_dir / 'Desktop'
print('The Desktop is at: {:}'.format(desktop_dir))

vis_depth_channel(qresults, desktop_dir / 'es_csi300_d_vs_c.pdf')

The Desktop is at: /Users/xuanyidong/Desktop
There are 48 qlib-results
